## Prog 2: 0915 Hrs Nifty Fno Status with option prices and option lot 
## Sheet 2 : Pchange comparison timestamp
## Sheet 3: Ranking system (To be corrected)

In [2]:
## 1. Program to fetch live F&O data for Nifty FnO stocks.
## 3. The program fetches the data for the following columns:
##    a. Symbol
##    b. Previous Close c. % Change d. LTP e. Industry f. Up Strike Price g. Up LTP h. Down Strike Price i. Down LTP j. Lot Size
## 4. The program saves the data to an Excel file and program applies color coding to the data based on the following conditions:
## 5. The program also saves the pChange data to a separate sheet 2 in the Excel file.
## 8. The program updates the ranks of the stocks in sheet 3 based on the pChange values.

import time
import requests
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill
from datetime import datetime, time as dt_time
import os
import jmespath
from memoization import cached
from datetime import datetime

# Initialize session object
session = requests.Session()
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
}

def get_new_session():
    global session
    session = requests.Session()
    headers['cookie'] = get_cookie()

def get_cookie():
    url = 'https://www.nseindia.com'
    response = session.get(url, headers=headers)
    cookies = response.cookies.get_dict()
    cookie_str = '; '.join([f"{key}={value}" for key, value in cookies.items()])
    return cookie_str

def fetch_data(url, retries=5, backoff_factor=0.3):
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 401:
                print("Cookie expired. Fetching a new cookie...")
                get_new_session()
            else:
                print(f"HTTP Error {response.status_code}: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}. Attempt {attempt + 1} of {retries}")
            if attempt < retries - 1:
                sleep_time = backoff_factor * (2 ** attempt)
                print(f"Retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            else:
                print("All retry attempts failed.")
    return None

def extract_data_post_open(data):
    data_list = []
    for item in data['data']:
        symbol = item['symbol']
        ltp = clean_value(item['lastPrice'])
        previous_close = clean_value(item['previousClose'])
        p_change = clean_value(item['pChange'])
        industry = item['meta'].get('industry', 'Unknown')  # Use .get() to provide a default value
        data_list.append([symbol, previous_close, p_change, ltp, industry])
    df = pd.DataFrame(data_list, columns=['symbol', 'Pr.Close', 'pChange', 'LTP', 'industry'])
    return df

def clean_value(value):
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return float(value)

def find_nearest_strike_prices(option_chain, ltp):
    strikes = [record['strikePrice'] for record in option_chain['records']['data']]
    strikes = sorted(strikes)
    nearest_strike_up = min([strike for strike in strikes if strike > ltp], default=None)
    nearest_strike_down = max([strike for strike in strikes if strike < ltp], default=None)
    return nearest_strike_up, nearest_strike_down

def get_option_details(option_chain, strike_price, option_type):
    for record in option_chain['records']['data']:
        if record['strikePrice'] == strike_price:
            if option_type == 'CE' and 'CE' in record:
                return record['CE']['lastPrice']
            elif option_type == 'PE' and 'PE' in record:
                return record['PE']['lastPrice']
    return None

def fetch_option_chain(symbol, retries=5, backoff_factor=0.3):
    url = f'https://www.nseindia.com/api/option-chain-equities?symbol={symbol}'
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 401:
                print("Cookie expired. Fetching a new cookie...")
                get_new_session()
            else:
                print(f"HTTP Error {response.status_code}: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}. Attempt {attempt + 1} of {retries}")
            if attempt < retries - 1:
                sleep_time = backoff_factor * (2 ** attempt)
                print(f"Retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            else:
                print("All retry attempts failed.")
    return None

@cached(ttl=30 * 24 * 3600)
def get_lot_sizes_dhan():
    res = requests.post(
        "https://open-web-scanx.dhan.co/scanx/allfut",
        json={"Data": {"Seg": 2, "Instrument": "FUT", "Count": 200, "Page_no": 1, "ExpCode": -1}},
        headers={"content-type": "application/json; charset=UTF-8"}
    )
    lots = jmespath.search("data.list[*].[sym, fo_dt[0].lot_type]", res.json())
    result = {x[0]: int(x[1].split()[0]) for x in lots}
    return result

def add_options_data(df):
    lot_size_data = get_lot_sizes_dhan()
    updated_data = []
    for index, row in df.iterrows():
        symbol = row['symbol']
        ltp = row['LTP']
        option_chain = fetch_option_chain(symbol)
        if option_chain:
            up_stk_pr, down_stk_pr = find_nearest_strike_prices(option_chain, ltp)
            up_ltp = get_option_details(option_chain, up_stk_pr, 'CE')
            down_ltp = get_option_details(option_chain, down_stk_pr, 'PE')
            lot_size = lot_size_data.get(symbol, None)
            updated_data.append([symbol, row['Pr.Close'], row['pChange'], ltp, row['industry'], up_stk_pr, up_ltp, down_stk_pr, down_ltp, lot_size])
    updated_df = pd.DataFrame(updated_data, columns=['symbol', 'Pr.Close', 'pChange', 'LTP', 'industry', 'Up_Stk_Pr', 'Up_LTP', 'Down_Stk_Pr', 'Down_LTP', 'Lot_Size'])
    return updated_df

def apply_styles(ws, n=5, threshold_high=3, threshold_mid=1):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['E2']

    # Identify all pChange columns
    pchange_columns = [cell.column_letter for cell in ws[1] if cell.value is not None and 'pChange' in cell.value]

    # Apply color coding to pChange columns
    for i, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
        for cell in row:
            if cell.column_letter in pchange_columns:
                if cell.value is not None:
                    if cell.value > 0:
                        if cell.value > threshold_high:
                            cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Bright green
                        elif cell.value > threshold_mid:
                            cell.fill = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")  # Medium green
                        else:
                            cell.fill = PatternFill(start_color="99FF99", end_color="99FF99", fill_type="solid")  # Light green
                    elif cell.value < 0:
                        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

            # Apply new color coding to every nth row
            if (i - 1) % n == 0:
                cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue color

def save_to_excel(df, filename):
    start_time = time.time()
    if os.path.exists(filename):
        wb = load_workbook(filename)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = 'F&O Data'

    # Write the main data to the active sheet
    for r_idx, row in df.iterrows():
        for c_idx, value in enumerate(row):
            ws.cell(row=r_idx + 2, column=c_idx + 1, value=value)
    for col_num, column_title in enumerate(df.columns, 1):
        ws.cell(row=1, column=col_num, value=column_title)

    # Apply styles including color coding
    apply_styles(ws)

    # Extract pChange values to a new sheet
    pchange_columns = [col for col in df.columns if 'pChange' in col]
    pchange_df = df[['symbol', 'Pr.Close', 'LTP'] + pchange_columns]

    # Create or update the new sheet for pChange values
    if 'pChange Data' in wb.sheetnames:
        ws_pchange = wb['pChange Data']
    else:
        ws_pchange = wb.create_sheet(title='pChange Data')

    for col_num, column_title in enumerate(pchange_df.columns, 1):
        ws_pchange.cell(row=1, column=col_num, value=column_title)

    for r_idx, row in pchange_df.iterrows():
        for c_idx, value in enumerate(row):
            ws_pchange.cell(row=r_idx + 2, column=c_idx + 1, value=value)

    # Apply styles to the pChange sheet
    apply_styles(ws_pchange)

    wb.save(filename)
    end_time = time.time()
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f"Data saved to {filename} at {timestamp}. Time taken: {end_time - start_time:.2f} seconds")

def append_timestamp_columns(existing_df, new_df):
    timestamp = datetime.now().strftime('%H_%M')
    if 'LTP' in new_df.columns:
        existing_df[f'LTP_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['LTP'])
    else:
        print(f"Column 'LTP' is missing in new_df. Available columns: {new_df.columns}")
        
    if 'pChange' in new_df.columns:
        existing_df[f'pChange_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['pChange'])
    else:
        print(f"Column 'pChange' is missing in new_df. Available columns: {new_df.columns}")
        
    if 'Up_Stk_Pr' in new_df.columns:
        existing_df[f'Up_Stk_Pr_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['Up_Stk_Pr'])
    else:
        print(f"Column 'Up_Stk_Pr' is missing in new_df. Available columns: {new_df.columns}")
        
    if 'Up_LTP' in new_df.columns:
        existing_df[f'Up_LTP_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['Up_LTP'])
    else:
        print(f"Column 'Up_LTP' is missing in new_df. Available columns: {new_df.columns}")
        
    if 'Down_Stk_Pr' in new_df.columns:
        existing_df[f'Down_Stk_Pr_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['Down_Stk_Pr'])
    else:
        print(f"Column 'Down_Stk_Pr' is missing in new_df. Available columns: {new_df.columns}")
        
    if 'Down_LTP' in new_df.columns:
        existing_df[f'Down_LTP_{timestamp}'] = existing_df['symbol'].map(new_df.set_index('symbol')['Down_LTP'])
    else:
        print(f"Column 'Down_LTP' is missing in new_df. Available columns: {new_df.columns}")
        
    return existing_df

def calculate_diff_pchange(existing_df):
    timestamp = datetime.now().strftime('%H_%M')
    pchange_columns = [col for col in existing_df.columns if col.startswith('pChange_')]
    
    # If only one pChange column exists, use it and the 'pChange' column for comparison
    if len(pchange_columns) == 1:
        existing_df[f'Diff_pChange_{timestamp}'] = existing_df[pchange_columns[0]] - existing_df['pChange']
    elif len(pchange_columns) > 1:
        last_pchange_col = pchange_columns[-1]
        prev_pchange_col = pchange_columns[-2]
        existing_df[f'Diff_pChange_{timestamp}'] = existing_df[last_pchange_col] - existing_df[prev_pchange_col]
    
    return existing_df

def update_ranks_in_excel(new_df, filename):
    wb = load_workbook(filename)
    
    # Check if 'Ranks' sheet exists
    if 'Ranks' in wb.sheetnames:
        ws_ranks = wb['Ranks']
        initial_df = pd.DataFrame(ws_ranks.values)
        if not initial_df.empty:
            initial_df.columns = initial_df.iloc[0]
            initial_df = initial_df[1:]
        else:
            # Initialize the DataFrame if 'Ranks' sheet is empty
            initial_df = pd.DataFrame(columns=['symbol', 'Pr.Close', 'pChange', 'LTP', 'Rank'])
    else:
        # Initialize the DataFrame if 'Ranks' sheet does not exist
        ws_ranks = wb.create_sheet(title='Ranks')
        initial_df = pd.DataFrame(columns=['symbol', 'Pr.Close', 'pChange', 'LTP', 'Rank'])

    # Add any new symbols from new_df to initial_df
    new_symbols = new_df[~new_df['symbol'].isin(initial_df['symbol'])]
    if not new_symbols.empty:
        new_symbols = new_symbols[['symbol']].copy()
        initial_df = pd.concat([initial_df, new_symbols], ignore_index=True)

    # Update the new pChange values and calculate new ranks
    for idx, row in initial_df.iterrows():
        symbol = row['symbol']
        if symbol in new_df['symbol'].values:
            initial_df.at[idx, 'pChange'] = new_df.loc[new_df['symbol'] == symbol, 'pChange'].values[0]
        else:
            initial_df.at[idx, 'pChange'] = None  # Or some default value if symbol is not found
    initial_df['New Rank'] = initial_df['pChange'].rank(method='first', ascending=False)

    # Add a new column for the current timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M')
    initial_df[f'Rank_{timestamp}'] = initial_df['New Rank']
    
    # Sort by initial Rank for display purposes
    initial_df = initial_df.sort_values('Rank').reset_index(drop=True)
    
    # Update the sheet with new ranks and timestamps
    for col_num, column_title in enumerate(initial_df.columns, 1):
        ws_ranks.cell(row=1, column=col_num, value=column_title)

    for r_idx, row in initial_df.iterrows():
        for c_idx, value in enumerate(row):
            ws_ranks.cell(row=r_idx + 2, column=c_idx + 1, value=value)
    
    wb.save(filename)

def job():
    current_time = datetime.now().time()
    if ((current_time >= dt_time(0, 15) and current_time <= dt_time(10, 40) and current_time.minute % 1 == 0) or 
        (current_time > dt_time(9, 40) and current_time < dt_time(10, 45) and current_time.minute % 3 == 0) or 
        (current_time > dt_time(10, 45) and current_time < dt_time(14, 0) and current_time.minute % 5 == 0) or 
        (current_time >= dt_time(14, 0) and current_time <= dt_time(15, 30) and current_time.minute % 1 == 0)):

        url = 'https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O'

        data = fetch_data(url)
        if data:
            df = extract_data_post_open(data)
            new_df = add_options_data(df)
            
            # print(f"new_df columns: {new_df.columns}")  # Debug statement
            
            # Generate a timestamp
            timestamp = datetime.now().strftime("%d%b")

            # Create the filename with the timestamp
            excel_filename = f'{timestamp}_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx'
            
            if os.path.exists(excel_filename):
                existing_df = pd.read_excel(excel_filename)
                existing_df = append_timestamp_columns(existing_df, new_df)
                existing_df = calculate_diff_pchange(existing_df)
                save_to_excel(existing_df, excel_filename)
                update_ranks_in_excel(existing_df, excel_filename)
            else:
                save_to_excel(new_df, excel_filename)
                update_ranks_in_excel(new_df, excel_filename)

def main():
    while True:
        job()
        time.sleep(60)  # Run the job every minute

if __name__ == "__main__":
    main()


Cookie expired. Fetching a new cookie...
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 09:57:30. Time taken: 0.09 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 09:59:27. Time taken: 0.14 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:01:19. Time taken: 0.19 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:03:19. Time taken: 0.32 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:05:26. Time taken: 0.29 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:07:30. Time taken: 0.34 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:09:30. Time taken: 0.56 seconds
Data saved to 09Aug_0915_3.1_Nifty_FnO_Rank_Sheet.xlsx at 10:11:18. Time taken: 0.49 seconds


KeyboardInterrupt: 